In [1]:
import os, sys

sys.path.append("../")

from typing import List, Dict, Union, Optional, Any, Tuple, Literal

import math
import random

import numpy as np

import torch
from torch.nn import functional as F

import lightning as L
from lightning.pytorch import seed_everything

from torch_geometric.data import HeteroData
from torch_geometric.loader import NeighborLoader
from torch_geometric.nn import conv, Sequential, summary
import torch_geometric.transforms as T
from torch_geometric.typing import EdgeType, NodeType

import torch_frame
from torch_frame import stype, NAStrategy
from torch_frame.nn import encoder
from torch_frame.nn import TabTransformerConv
from torch_frame.data import StatType


from db_transformer.nn import (
    BlueprintModel,
    EmbeddingTranscoder,
    SelfAttention,
    CrossAttentionConv,
    NodeApplied,
)
from db_transformer.nn.lightning import LightningWrapper
from db_transformer.nn.lightning.callbacks import BestMetricsLoggerCallback

from db_transformer.data.ctu_dataset import CTUDataset, CTU_REPOSITORY_DEFAULTS

from experiments.blueprint_instances import create_blueprint_model


%reload_ext autoreload
%autoreload 2

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
dataset = CTUDataset("Chess", data_dir="../datasets", force_remake=False)

data = dataset.build_hetero_data(force_rematerilize=False, no_text_emebedding=False)

n_total = data[dataset.defaults.target_table].y.shape[0]

data = T.RandomNodeSplit(split="train_rest", num_val=int(0.30 * n_total), num_test=0)(data)

Building data:   0%|          | 0/2 [00:00<?, ?it/s]

Table game has stypes:
	categorical: ['b1', 'b2', 'b3', 'b4', 'event', 'opening', 'site', 'w1', 'w2', 'w3', 'w4']
	timestamp: ['event_date']
	numerical: ['BlackElo', 'whiteElo']
	embedding: ['ECO', 'b10', 'b5', 'b6', 'b7', 'b8', 'b9', 'black', 'round', 'w10', 'w5', 'w6', 'w7', 'w8', 'w9', 'white']
Table opening has stypes:
	categorical: ['b1', 'b2', 'b3', 'b4', 'code', 'name', 'w1', 'w2', 'w3', 'w4']
	embedding: ['variation']


In [3]:
target = dataset.defaults.target

seed_everything(42)

num_neighbors = {
    edge_type: [30] * 5 for edge_type in data.collect("edge_index", allow_empty=True).keys()
}

train_loader = NeighborLoader(
    data,
    num_neighbors=num_neighbors,
    batch_size=512,
    input_nodes=(target[0], data[target[0]].train_mask),
    subgraph_type="bidirectional",
    shuffle=True,
)

val_loader = NeighborLoader(
    data,
    num_neighbors=num_neighbors,
    batch_size=512,
    input_nodes=(target[0], data[target[0]].val_mask),
    subgraph_type="bidirectional",
    shuffle=True,
)

Seed set to 42


In [4]:
sample: HeteroData = next(iter(train_loader)).to(device)

y: torch.Tensor = sample[target[0]].y
# print(y.unique(return_counts=True))
print(y.min(), y.max())
data.collect("edge_index").keys()

tensor(0, device='cuda:0') tensor(2, device='cuda:0')


dict_keys([('game', 'opening_id', 'opening'), ('opening', 'rev_opening_id', 'game')])

In [34]:
edge_types = list(data.collect("edge_index", allow_empty=True).keys())

model = create_blueprint_model(
    "mlp",
    dataset.defaults,
    {node: tf.col_names_dict for node, tf in data.collect("tf").items() if tf.num_rows > 0},
    edge_types,
    data.collect("col_stats"),
    dict(
        embed_dim=64,
        gnn_layers=0,
        mlp_dims=[64, 64],
        residual=False,
        batch_norm=True,
        dropout=0,
    ),
).to(device)

print(
    summary(
        model,
        sample.collect("tf"),
        sample.collect("edge_index", allow_empty=True),
        max_depth=10,
    )
)

+----------------------------------------------------------+---------------+----------------+----------+
| Layer                                                    | Input Shape   | Output Shape   | #Param   |
|----------------------------------------------------------+---------------+----------------+----------|
| BlueprintModel                                           |               | [277, 3]       | 114,883  |
| ├─(embedder)Sequential                                   |               |                | 57,216   |
| │    └─(module_0)DBEmbedder                              |               |                | 57,216   |
| │    │    └─(table_embedders)ModuleDict                  | --            | --             | 57,216   |
| │    │    │    └─(game)TableEmbedder                     |               | [277, 13, 64]  | 15,296   |
| │    │    │    │    └─(encoder_dict)ModuleDict           | --            | --             | 15,296   |
| │    │    │    │    │    └─(categorical)EmbeddingEnco

In [35]:
lightning_model = LightningWrapper(
    model, dataset.defaults.target_table, lr=0.0001, task_type=dataset.defaults.task
).to(device)
metric = "acc"


trainer = L.Trainer(
    accelerator=device.type,
    devices=1,
    deterministic=False,
    max_epochs=1000,
    max_steps=-1,
    enable_checkpointing=False,
    logger=False,
    callbacks=[
        BestMetricsLoggerCallback(
            monitor=f"val_{metric}",
            cmp="max",
            metrics=[
                # "train_loss",
                # "val_loss",
                # "test_loss",
                # f"train_{metric}",
                f"val_{metric}",
                f"test_{metric}",
            ],
        ),
    ],
)

trainer.fit(lightning_model, train_loader, val_dataloaders=val_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name        | Type             | Params
-------------------------------------------------
0 | model       | BlueprintModel   | 114 K 
1 | loss_module | CrossEntropyLoss | 0     
-------------------------------------------------
114 K     Trainable params
0         Non-trainable params
114 K     Total params
0.460     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=1000` reached.
